## data

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
cd /content/drive/MyDrive/3033proj

/content/drive/.shortcut-targets-by-id/15vNI9If7Er0ikh22tgyBff1Q88-gfcL-/3033proj


In [ ]:
import keras
import os
import numpy as np

class Datasets(object):
    def __init__(self,data_name):
        self.data_name=data_name
        self.datasets = self.getData(self)
        self.final_data = []
        self.convert_data_format()


    def gen(self):
        # np.random.shuffle(self.final_data)

        images = []
        age_labels = []
        gender_labels = []
        race_labels=[]

        for i in range(len(self.final_data)):
            image, age, gender,race = self.final_data[i]
            images.append(image)
            age_labels.append(age)
            gender_labels.append(gender)
            race_labels.append(race)
    
        age_labels = keras.utils.to_categorical(age_labels, num_classes=NUM_AGE_CLASSES)
        gender_labels = keras.utils.to_categorical(gender_labels, num_classes=2)
        race_labels = keras.utils.to_categorical(race_labels, num_classes=5)

        return images, age_labels, gender_labels, race_labels

    @staticmethod
    def getData(self):
        print('Loading age image...')
        if self.data_name=='data_3_utk':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_3_utk.npy'), allow_pickle=True) 
        if self.data_name=='data_5_utk':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_5_utk.npy'), allow_pickle=True) 
        if self.data_name=='data_10_utk':
          data = np.load(os.path.join(os.getcwd(), '/content/drive/MyDrive/3033proj/data_10_utk.npy'), allow_pickle=True)
        np.random.shuffle(data)
        all_data = []
        # random select 20000 data
        for i in range(20000):  ### number of samples
            all_data.append(data[i])
        print('Number of age data:', str(len(all_data)))
        return all_data

    def convert_data_format(self):
        # Age datasets:
        for i in range(len(self.datasets)):
            image = self.datasets[i][0] / 255.0
            age_labels = self.datasets[i][1]
            gender_labels = self.datasets[i][2]
            race_labels= self.datasets[i][3]

            self.final_data.append((image, age_labels, gender_labels,race_labels))

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import optimizers
from keras.utils.generic_utils import get_custom_objects
from keras.layers import GlobalMaxPool2D, Dropout, Dense, Activation, BatchNormalization
from keras.models import Model
import numpy as np
from keras.backend import sigmoid
from efficientnet.keras import EfficientNetB4 as NetB4
from efficientnet.keras import EfficientNetB4 as NetB5
from efficientnet.keras import EfficientNetB4 as NetB6
from efficientnet.keras import EfficientNetB4 as NetB7

## set parameters

In [ ]:
IMAGE_SIZE = 56
BATCH_SIZE = 256
WEIGHT_INIT = 0.08
NUM_AGE_CLASSES = 3
NUM_GENDER_CLASSES = 2
DROPOUT_RATE = 0.2
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
NUM_EPOCHS = 40
DECAY_LR_RATE = 0.9
NUM_SAMPLE=20000

## model

In [ ]:
import pandas as pd
class SwishActivation(Activation):
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta=1):
    return x * sigmoid(beta * x)

class Efficient_Net(object):
    def __init__(self, model_name, data_name, trainable=True):
        self.trainable = trainable
        self.model_name=model_name
        self.data_name=data_name
        if self.trainable: 
            self.train_data = Datasets(data_name=self.data_name)
        self.model = self.build_model(self.model_name)

        # Compile the model
        losses = {
            "age_output": "categorical_crossentropy",
            "gender_output": "categorical_crossentropy",
             "race_output": "categorical_crossentropy"

        }

        opt = optimizers.Adam(1e-3)
        self.model.compile(loss=losses, optimizer=opt, metrics=['acc'])

        # Train the part you added
        if self.trainable:
            self.model.summary()

    @staticmethod
    def build_age_branch(x):
        # Output age branch
        predictions_age = Dense(NUM_AGE_CLASSES, activation="softmax", name='age_output')(x)

        return predictions_age

    @staticmethod
    def build_gender_branch(x):
        # Output gender branch
        predictions_gender = Dense(2, activation="softmax", name='gender_output')(x)

        return predictions_gender

    def build_race_branch(self,x):
        # Output race branch
        predictions_race = Dense(5, activation="softmax", name='race_output')(x)

        return predictions_race

    def build_model(self,model_name):
        get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

        # Model
        if model_name=='age_gender_race_B4':
          model = NetB4(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='age_gender_race_B5':
          model = NetB5(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='age_gender_race_B6':
          model = NetB6(weights='imagenet', include_top=False, input_shape=input_shape)
        if model_name=='age_gender_race_B7':
          model = NetB7(weights='imagenet', include_top=False, input_shape=input_shape)
        

        # Adding 2 fully-connected layers to B4.
        x = model.output
        x = BatchNormalization()(x)
        x = GlobalMaxPool2D(name='gap1')(x)
        x = Dropout(DROPOUT_RATE, name='dropout1')(x)

        # Output layer
        predictions_age = self.build_age_branch(x)
        predictions_gender = self.build_gender_branch(x)
        predictions_race = self.build_race_branch(x)
        model_final = Model(inputs=model.input, outputs=[predictions_age, predictions_gender,predictions_race])

        return model_final

    def train(self):
        # reduce learning rate
        reduce_lr = ReduceLROnPlateau(monitor='val_age_output_acc', factor=DECAY_LR_RATE, patience=5, verbose=1, )
        # Model Checkpoint
        # cpt_save = ModelCheckpoint( '/content/drive/MyDrive/3033proj/weights/weight_{}.h5'.format(model_name+data_name), save_best_only=True, monitor='val_race_output_acc', mode='max')

        print("Training......")
        trainX, trainAgeY, trainGenderY ,trainRaceY= self.train_data.gen()
        trainX = np.array(trainX)


        history=self.model.fit(trainX, {"age_output": trainAgeY, "gender_output": trainGenderY, "race_output": trainRaceY}, validation_split=0.2,
                       callbacks=reduce_lr, verbose=1, epochs=40, shuffle=True,
                       batch_size=256)
        
        dir ='/content/drive/MyDrive/3033proj/model_log/'
        log_path = dir+self.model_name+self.data_name+'_log.csv'
        hist = pd.DataFrame(history.history)
        hist.to_csv(log_path)

### 'age_gender_race_B4', 'data_3'

In [ ]:
NUM_AGE_CLASSES=3
model = Efficient_Net('age_gender_race_B4', 'data_3_utk', trainable=True)

In [ ]:
model.train('age_gender_race_B4', 'data_3_utk')

### 'age_gender_race_B5', 'data_3_utk'

In [ ]:
NUM_AGE_CLASSES=3
model = Efficient_Net('age_gender_race_B5', 'data_3_utk', trainable=True)

In [ ]:
model.train()

### 'age_gender_race_B6', 'data_3_utk'

In [ ]:
NUM_AGE_CLASSES=3
model = Efficient_Net('age_gender_race_B6', 'data_3_utk', trainable=True)

In [ ]:
model.train()

### 'age_gender_race_B7', 'data_3_utk'

In [ ]:
NUM_AGE_CLASSES=3
model = Efficient_Net('age_gender_race_B7', 'data_3_utk', trainable=True)

In [ ]:
model.train()

### age_gender_race_B4', 'data_5_utk'

In [ ]:
NUM_AGE_CLASSES=5
model = Efficient_Net('age_gender_race_B4', 'data_5_utk', trainable=True)

In [ ]:
model.train()

### age_gender_race_B5', 'data_5_utk'

In [ ]:
NUM_AGE_CLASSES=5
model = Efficient_Net('age_gender_race_B5', 'data_5_utk', trainable=True)

In [ ]:
model.train()

### age_gender_race_B6', 'data_5_utk'

In [ ]:
NUM_AGE_CLASSES=5
model = Efficient_Net('age_gender_race_B6', 'data_5_utk', trainable=True)

In [ ]:
model.train()

### 'age_gender_race_B7', 'data_5_utk'

In [ ]:
NUM_AGE_CLASSES=5
model = Efficient_Net('age_gender_race_B7', 'data_5_utk', trainable=True)

In [ ]:
model.train()

### 'age_gender_race_B4', 'data_10_utk',

In [ ]:
NUM_AGE_CLASSES=10
model = Efficient_Net('age_gender_race_B4', 'data_10_utk', trainable=True)

In [ ]:
model.train()

### age_gender_race_B5', 'data_10_utk'

In [ ]:
NUM_AGE_CLASSES=10
model = Efficient_Net('age_gender_race_B5', 'data_10_utk', trainable=True)

In [ ]:
model.train()

### 'age_gender_race_B6', 'data_10_utk',

In [ ]:
NUM_AGE_CLASSES=10
model = Efficient_Net('age_gender_race_B6', 'data_10_utk', trainable=True)

In [ ]:
model.train()

### 'age_gender_race_B7', 'data_10_utk'

In [ ]:
NUM_AGE_CLASSES=10
model = Efficient_Net('age_gender_race_B7', 'data_10_utk', trainable=True)

In [ ]:
model.train()